# Rap Music Analysis

In [1]:
import urllib.request
import urllib.parse
import urllib.error
from urllib.request import Request, urlopen
import requests

from bs4 import BeautifulSoup

import pandas as pd
import json

import string

In [2]:
# Create an empty DataFrame
column_names = ['Title','Artist','Lyrics','Comments']
df = pd.DataFrame(columns = column_names)

In [6]:
def request_artist_info(artist_name, page):
    
    base_url = 'https://api.genius.com'
    
    token = input('Enter your Genius API token:')
    
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response 

In [19]:
def request_artist_info(artist_name, page):
    
    import getpass
    base_url = 'https://api.genius.com'
    token = str(getpass.getpass('Enter your Genius API token:'))
    
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response 

In [20]:
response = request_artist_info('Drake', 1)
json = response.json()

Enter your Genius API token:········


In [22]:
json = response.json()

artist = []
song_name = []

for hit in json['response']['hits']: 
    artist.append(hit['result']['primary_artist']['name'])
    song_name.append(hit['result']['title'])

KeyError: 'response'

In [9]:
df['Title']=song_name
df['Artist'] = artist

In [10]:
def request_song_url(artist_name, song_cap):
    
    """ This function populates a list with urls to the songs by a chosen artist. 

    Args: 
        artist_name(str): name of the artist
        song_cap(int): number of urls/songs you want 
        
    Returns: 
        list of strings, each string is a url to a song by the artist
    
    """
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()       
        song_info = []
        
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
    
    return songs

In [11]:
urls = request_song_url('drake',20)

Enter your Genius API token:VetJWKIjYbo8ujjg8V_PTQnqAkY8xN1fbajgaaTiON0rkgWFuI6X47dXPKCaZQLM


In [12]:
urls

['https://genius.com/Drake-gods-plan-lyrics',
 'https://genius.com/Drake-in-my-feelings-lyrics',
 'https://genius.com/Drake-hotline-bling-lyrics',
 'https://genius.com/Drake-one-dance-lyrics',
 'https://genius.com/Drake-hold-on-were-going-home-lyrics',
 'https://genius.com/Drake-know-yourself-lyrics',
 'https://genius.com/Drake-back-to-back-lyrics',
 'https://genius.com/Drake-all-me-lyrics',
 'https://genius.com/Drake-fake-love-lyrics',
 'https://genius.com/Drake-from-time-lyrics',
 'https://genius.com/Drake-0-to-100-the-catch-up-lyrics',
 'https://genius.com/Drake-started-from-the-bottom-lyrics',
 'https://genius.com/Drake-the-motto-lyrics',
 'https://genius.com/Drake-pound-cake-paris-morton-music-2-lyrics',
 'https://genius.com/Drake-and-future-jumpman-lyrics',
 'https://genius.com/Drake-hyfr-lyrics',
 'https://genius.com/Drake-marvins-room-lyrics',
 'https://genius.com/Drake-nice-for-what-lyrics',
 'https://genius.com/Drake-passionfruit-lyrics',
 'https://genius.com/Drake-childs-pla

In [13]:
counter = 0

for url in urls: 
    
    req = Request(url, headers = {"User-Agent" : "Mozilla/5.0"})
    webpage = urlopen(req).read()

    # Create a BeautifulSoup object 
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    songs = {}
    songs['lyrics'] = [];
    songs['comments'] = [];

    # Extract user comments on the song
    for div in soup.findAll('div',attrs={'class':'rich_text_formatting'}):
        comments = div.text.strip().split("\n")
        
        for comment in comments: 
            if comment != "":
                songs['comments'].append(comment);
                df['Comments'].iloc[counter] = songs['comments']
        
    # Extract the lyrics of the song
    for div in soup.findAll('div', attrs = {'class':'lyrics'}):
        songs['lyrics'].append(div.text.strip().split("\n"));
        df['Lyrics'].iloc[counter] = songs['lyrics'][0]
    
    counter +=1

In [ ]:
df

In [ ]:
def remove_punctuation(str1): 
    
    for char in str1.lower():
        
        if char in string.punctuation: 
            
            str1 = str1.replace(char,"")          
    
    return str1

In [ ]:
lyric_values = df['Lyrics']

size = len(lyric_values)  # size holds the number of rows
n=-1 # we want to start from cell 0, the first increment would make the -1 a 0

lst_words = []

while True: 
    
    n += 1  
    if n == size: 
        break 
    
    wrds = [] # will hold the list of words in each cell 

    # inspect the next cell
    print ("-" * 40, "cell " + str(n), "-" * 40)
    text = lyric_values[n][1]
    text_str = str(text).lower()
    no_punct_str = remove_punctuation(text_str)
    
    for word in no_punct_str.split(): 
        wrds.append(word)
        
    lst_words.append(len(wrds))

    print ("The text in this cell:\n\n", text, "\n")
    print("The number of words in this cell:\n",len(wrds),'\n\n')

In [ ]:
df.insert(4,'total_words', pd.Series(lst_words))

In [ ]:
df